In [1]:
import numpy as np
import unittest

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical as keras_to_categorical

from src.core import Tensor
from src.functions import Linear, ReLU, Softmax, MeanSquaredError, CategoricalCrossEntropy
from src.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Flatten
from src.model import Sequential
from src.optimization import Optimizer, Adam
from src.tests import TestCNNLayer, TestCNNModel, TestLoadKerasWeights

### **Tensor Test Case**

In [2]:
# Basic operations
a = Tensor(np.array([1,2]))
b = Tensor(np.array([3,4]))
c = a + b
d = a - b
e = a * b
f = a / b

print(a)
print(b)
print(c)
print(d)
print(e)
print(f)

Tensor(data=[1. 2.], grad=[0. 0.], op='None')
Tensor(data=[3. 4.], grad=[0. 0.], op='None')
Tensor(data=[4. 6.], grad=[0. 0.], op='+')
Tensor(data=[-2. -2.], grad=[0. 0.], op='+')
Tensor(data=[3. 8.], grad=[0. 0.], op='*')
Tensor(data=[0.33333333 0.5       ], grad=[0. 0.], op='*')


In [3]:
# Activation function and loss function
a = Tensor(np.array([1,2]))
b = a.compute_activation(Linear)
c = a.compute_activation(ReLU)
d = b.compute_loss(Tensor(np.array([3,4])), MeanSquaredError)
e = c.compute_loss(Tensor(np.array([3,4])), MeanSquaredError)

print(a)
print(b)
print(c)
print(d)
print(e)

Tensor(data=[1. 2.], grad=[0. 0.], op='None')
Tensor(data=[1. 2.], grad=[0. 0.], op='Linear')
Tensor(data=[1. 2.], grad=[0. 0.], op='ReLU')
Tensor(data=[4.], grad=[0.], op='MeanSquaredError')
Tensor(data=[4.], grad=[0.], op='MeanSquaredError')


In [4]:
# Automatic differentiation
a = Tensor(np.array([1,2]))
b = Tensor(np.array([3,4]))
c = a + b
d = a - b
e = c * d
f = e.compute_activation(Linear)
g = e.compute_activation(ReLU)
h = f.compute_loss(np.array([1,1]), MeanSquaredError)
i = g.compute_loss(np.array([1,1]), MeanSquaredError)

print("---------- Before backpropagation ----------")
print(a)
print(b)
print(c)
print(d)
print(e)
print(f)
print(g)
print(h)
print(i)

h.backward()
i.backward()

print("\n---------- After backpropagation ----------")
print(a)
print(b)
print(c)
print(d)
print(e)
print(f)
print(g)
print(h)
print(i)

---------- Before backpropagation ----------
Tensor(data=[1. 2.], grad=[0. 0.], op='None')
Tensor(data=[3. 4.], grad=[0. 0.], op='None')
Tensor(data=[4. 6.], grad=[0. 0.], op='+')
Tensor(data=[-2. -2.], grad=[0. 0.], op='+')
Tensor(data=[ -8. -12.], grad=[0. 0.], op='*')
Tensor(data=[ -8. -12.], grad=[0. 0.], op='Linear')
Tensor(data=[0. 0.], grad=[0. 0.], op='ReLU')
Tensor(data=[125.], grad=[0.], op='MeanSquaredError')
Tensor(data=[1.], grad=[0.], op='MeanSquaredError')

---------- After backpropagation ----------
Tensor(data=[1. 2.], grad=[ -54. -156.], op='None')
Tensor(data=[3. 4.], grad=[198. 390.], op='None')
Tensor(data=[4. 6.], grad=[36. 52.], op='+')
Tensor(data=[-2. -2.], grad=[ -72. -156.], op='+')
Tensor(data=[ -8. -12.], grad=[ -9. -13.], op='*')
Tensor(data=[ -8. -12.], grad=[ -9. -13.], op='Linear')
Tensor(data=[0. 0.], grad=[-1. -1.], op='ReLU')
Tensor(data=[125.], grad=[1.], op='MeanSquaredError')
Tensor(data=[1.], grad=[1.], op='MeanSquaredError')


In [5]:
## Simulation of one layer with two neurons (h1 and h2)

# Initial values
x = Tensor(np.array([1, 2, 3]), tensor_type="input")            # input, x[0] is always 1
y = np.array([16, 14])                                          # correct class / y_true
wh1 = Tensor(np.array([2, 3, 4]), tensor_type="weight")         # weights of neuron h1, wh1[0] = b1 (bias)
wh2 = Tensor(np.array([3, 4, 5]), tensor_type="weight")         # weights of neuron h2, wh2[0] = b2 (bias)

# Calculate net
wh1_x = wh1 * x
wh2_x = wh2 * x
net1 = wh1_x.sum()
net2 = wh2_x.sum()

# Calculate output
o1 = net1.compute_activation(ReLU)
o2 = net2.compute_activation(ReLU)

# Calculate loss
output = o1.concat([o2])
loss = output.compute_loss(y, MeanSquaredError)

print("---------- Before backpropagation ----------")
print(wh1)
print(wh2)
print(wh1_x)
print(wh2_x)
print(net1)
print(net2)
print(o1)
print(o2)
print(output)
print(loss)

# Initiate automated differentiation
loss.backward()

print("\n---------- After backpropagation ----------")
print(wh1)
print(wh2)
print(wh1_x)
print(wh2_x)
print(net1)
print(net2)
print(o1)
print(o2)
print(output)
print(loss)


---------- Before backpropagation ----------
Tensor(data=[2. 3. 4.], grad=[0. 0. 0.], op='None', type=weight)
Tensor(data=[3. 4. 5.], grad=[0. 0. 0.], op='None', type=weight)
Tensor(data=[ 2.  6. 12.], grad=[0. 0. 0.], op='*')
Tensor(data=[ 3.  8. 15.], grad=[0. 0. 0.], op='*')
Tensor(data=[20.], grad=[0.], op='sum(axis=None, keepdims=False)')
Tensor(data=[26.], grad=[0.], op='sum(axis=None, keepdims=False)')
Tensor(data=[20.], grad=[0.], op='ReLU')
Tensor(data=[26.], grad=[0.], op='ReLU')
Tensor(data=[20. 26.], grad=[0. 0.], op='concatenate(axis=0)')
Tensor(data=[80.], grad=[0.], op='MeanSquaredError')

---------- After backpropagation ----------
Tensor(data=[2. 3. 4.], grad=[ 4.  8. 12.], op='None', type=weight)
Tensor(data=[3. 4. 5.], grad=[12. 24. 36.], op='None', type=weight)
Tensor(data=[ 2.  6. 12.], grad=[4. 4. 4.], op='*')
Tensor(data=[ 3.  8. 15.], grad=[12. 12. 12.], op='*')
Tensor(data=[20.], grad=[4.], op='sum(axis=None, keepdims=False)')
Tensor(data=[26.], grad=[12.], op=

In [6]:
## Simulation of 3-layered (excluding input layer) network with n = [3, 5, 4] number of neurons

# Initial values
x = Tensor(np.array([1, 2, 3]), tensor_type="input")            
y = np.array([50, 64, 62, 55])                                          
wh1 = Tensor(np.array([2, 3, 4]), tensor_type="weight")
wh2 = Tensor(np.array([3, 4, 5]), tensor_type="weight")
wh3 = Tensor(np.array([4, 5, 6]), tensor_type="weight")
wh4 = Tensor(np.array([2, 3, 4, 5]), tensor_type="weight")
wh5 = Tensor(np.array([3, 4, 5, 6]), tensor_type="weight")
wh6 = Tensor(np.array([4, 5, 6, 7]), tensor_type="weight")
wh7 = Tensor(np.array([5, 6, 7, 8]), tensor_type="weight")
wh8 = Tensor(np.array([6, 7, 8, 9]), tensor_type="weight")
wh9 = Tensor(np.array([2, 3, 4, 5, 6, 7]), tensor_type="weight")
wh10 = Tensor(np.array([3, 4, 5, 6, 7, 8]), tensor_type="weight")
wh11 = Tensor(np.array([4, 5, 6, 7, 8, 9]), tensor_type="weight")
wh12 = Tensor(np.array([5, 6, 7, 8, 9, 10]), tensor_type="weight")


# Layer 1
wh1_x = wh1 * x
wh2_x = wh2 * x
wh3_x = wh3 * x
net1 = wh1_x.sum()
net2 = wh2_x.sum()
net3 = wh3_x.sum()
o1 = net1.compute_activation(ReLU)
o2 = net2.compute_activation(ReLU)
o3 = net3.compute_activation(ReLU)
output_l1 = o1.concat([o2, o3])

# Layer 2
input_l2 = output_l1.add_x0()
wh4_l2 = wh4 * input_l2
wh5_l2 = wh5 * input_l2
wh6_l2 = wh6 * input_l2
wh7_l2 = wh7 * input_l2
wh8_l2 = wh8 * input_l2
net4 = wh4_l2.sum()
net5 = wh5_l2.sum()
net6 = wh6_l2.sum()
net7 = wh7_l2.sum()
net8 = wh8_l2.sum()
o4 = net4.compute_activation(ReLU)
o5 = net5.compute_activation(ReLU)
o6 = net6.compute_activation(ReLU)
o7 = net7.compute_activation(ReLU)
o8 = net8.compute_activation(ReLU)
output_l2 = o4.concat([o5, o6, o7, o8])

# Layer 3
input_l3 = output_l2.add_x0()
wh9_l3 = wh9 * input_l3
wh10_l3 = wh10 * input_l3
wh11_l3 = wh11 * input_l3
wh12_l3 = wh12 * input_l3
net9 = wh9_l3.sum()
net10 = wh10_l3.sum()
net11 = wh11_l3.sum()
net12 = wh12_l3.sum()
o9 = net9.compute_activation(ReLU)
o10 = net10.compute_activation(ReLU)
o11 = net11.compute_activation(ReLU)
o12 = net12.compute_activation(ReLU)
output_l3 = o9.concat([o10, o11, o12])

# Compute loss
loss = output_l3.compute_loss(y, MeanSquaredError)

# Backpropagation
loss.backward()

print("============ Layer 1 ============")
print(wh1_x)
print(wh2_x)
print(wh3_x)
print(net1)
print(net2)
print(net3)
print(o1)
print(o2)
print(o3)
print(output_l1)
print("\n============ Layer 2 ============")
print(input_l2)
print(wh4_l2)
print(wh5_l2)
print(wh6_l2)
print(wh7_l2)
print(wh8_l2)
print(net4)
print(net5)
print(net6)
print(net7)
print(net8)
print(o4)
print(o5)
print(o6)
print(o7)
print(o8)
print(output_l2)
print("\n============ Layer 3 ============")
print(input_l3)
print(wh9_l3)
print(wh10_l3)
print(wh11_l3)
print(wh12_l3)
print(net9)
print(net10)
print(net11)
print(net12)
print(o9)
print(o10)
print(o11)
print(o12)
print(output_l3)
print(loss)

============ Layer 1 ============
Tensor(data=[ 2.  6. 12.], grad=[5831915. 5831915. 5831915.], op='*')
Tensor(data=[ 3.  8. 15.], grad=[6932435. 6932435. 6932435.], op='*')
Tensor(data=[ 4. 10. 18.], grad=[8032955. 8032955. 8032955.], op='*')
Tensor(data=[20.], grad=[5831915.], op='sum(axis=None, keepdims=False)')
Tensor(data=[26.], grad=[6932435.], op='sum(axis=None, keepdims=False)')
Tensor(data=[32.], grad=[8032955.], op='sum(axis=None, keepdims=False)')
Tensor(data=[20.], grad=[5831915.], op='ReLU')
Tensor(data=[26.], grad=[6932435.], op='ReLU')
Tensor(data=[32.], grad=[8032955.], op='ReLU')
Tensor(data=[20. 26. 32.], grad=[5831915. 6932435. 8032955.], op='concatenate(axis=0)')

============ Layer 2 ============
Tensor(data=[ 1. 20. 26. 32.], grad=[4731395. 5831915. 6932435. 8032955.], op='add_x0')
Tensor(data=[  2.  60. 104. 160.], grad=[154241. 154241. 154241. 154241.], op='*')
Tensor(data=[  3.  80. 130. 192.], grad=[187172.5 187172.5 187172.5 187172.5], op='*')
Tensor(data=[  

In [7]:
# Simulation for input with 3 features and batch_size = 3
x = Tensor(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), tensor_type="input")
y = np.array([
    [50, 60, 70, 80, 90],
    [55, 65, 75, 85, 95],
    [52, 62, 72, 82, 92]
])

# Layer 1
wh1_x = wh1 * x
wh2_x = wh2 * x
wh3_x = wh3 * x
net1 = wh1_x.sum()
net2 = wh2_x.sum()
net3 = wh3_x.sum()
o1 = net1.compute_activation(ReLU)
o2 = net2.compute_activation(ReLU)
o3 = net3.compute_activation(ReLU)
output_l1 = o1.concat([o2, o3])

# Layer 2
input_l2 = output_l1.add_x0()
wh4_l2 = wh4 * input_l2
wh5_l2 = wh5 * input_l2
wh6_l2 = wh6 * input_l2
wh7_l2 = wh7 * input_l2
wh8_l2 = wh8 * input_l2
net4 = wh4_l2.sum()
net5 = wh5_l2.sum()
net6 = wh6_l2.sum()
net7 = wh7_l2.sum()
net8 = wh8_l2.sum()
o4 = net4.compute_activation(ReLU)
o5 = net5.compute_activation(ReLU)
o6 = net6.compute_activation(ReLU)
o7 = net7.compute_activation(ReLU)
o8 = net8.compute_activation(ReLU)
output_l2 = o4.concat([o5, o6, o7, o8])

# Compute loss
loss = output_l2.compute_loss(y, MeanSquaredError)

# Backpropagation
loss.backward()

### **CNN Test Case**

In [8]:
# Unit tests for Convolution layer
unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromTestCase(TestCNNLayer))

......


--- Grad Check: AveragePooling2DLayer ---
AveragePooling gradient check PASSED.

--- Grad Check: FlattenLayer ---
Flatten gradient check PASSED.

--- Forward Test Large: N=2,C_in=3,H=32,W=32,K=8,KS=(3, 3),S=(2, 2),P='valid',Bias=False,Act=None ---
Conv2d with name=conv2d_layer_1564793761232 is setting weights from keras
Weights:
[Tensor(data=[[[[-1.63451943e-02 -8.78518854e-04  2.22505160e-02]
   [-6.95829125e-03  2.15698706e-02  2.23479172e-02]
   [-1.64271386e-02  8.00077858e-03  8.90878105e-03]]

  [[ 1.30384768e-02  9.00684995e-03 -1.70153400e-03]
   [ 5.20809965e-03 -2.71902804e-03  6.00852708e-03]
   [ 8.14806599e-03  1.52535185e-02  1.08457816e-04]]

  [[ 2.28644251e-02 -4.73735430e-03 -8.39647517e-03]
   [-7.82346020e-03  9.71022480e-03 -1.15813546e-02]
   [-8.01975059e-03  5.49743088e-03 -5.01125700e-03]]]


 [[[ 6.12340447e-03 -6.46699594e-03 -1.37464861e-02]
   [ 1.07505000e-03 -2.11155013e-03 -5.06961400e-04]
   [-5.60894767e-03 -1.84763611e-03 -3.41784216e-03]]

  [[-1.05

.

Forward Test Large completed in 0.0688 seconds.

--- Grad Check Large: N=1,C_in=3,H=10,W=10,K=2,KS=(3, 3),S=(1, 1),P='valid',Bias=False,Act=None ---
Conv2d with name=conv2d_layer_1564810356592 is setting weights from keras
Weights:
[Tensor(data=[[[[-0.00868136  0.09595572 -0.03770834]
   [ 0.00619841 -0.04395261 -0.01393114]
   [ 0.00216843 -0.0080194   0.02909097]]

  [[ 0.09057433  0.07706959 -0.04460096]
   [-0.04197175  0.04513542  0.03772076]
   [-0.00265418  0.01392951 -0.06777615]]

  [[ 0.06052429 -0.00907653  0.06205628]
   [ 0.02332313 -0.03349891 -0.11945563]
   [ 0.00195461  0.06371197  0.07387999]]]


 [[[-0.00212206  0.11097545 -0.11530074]
   [ 0.04319918 -0.03822113  0.0076847 ]
   [-0.0242132  -0.03926831  0.0038746 ]]

  [[ 0.02916971 -0.00700114  0.07037667]
   [-0.04159867  0.01155329 -0.13772568]
   [-0.02410976  0.10528705 -0.07840405]]

  [[-0.03270005  0.06971038  0.03566918]
   [ 0.00792673 -0.03058225  0.01109659]
   [ 0.01908384 -0.030806    0.0353184 ]]]], g

.

...

....

Numerical bias gradient calculated.
Bias gradient check PASSED (Large Input).
Grad Check Large completed in 0.0311 seconds.

--- Grad Check: MaxPooling2DLayer ---


..
----------------------------------------------------------------------
Ran 14 tests in 0.230s

OK


MaxPooling gradient check PASSED.

--- Grad Check: MaxPooling2DLayer (Overlapping) ---
MaxPooling overlapping gradient check PASSED.


<unittest.runner.TextTestResult run=14 errors=0 failures=0>

In [9]:
# Unit test for a full CNN model architecture
unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromTestCase(TestCNNModel))

.


--- TestSequentialModel: Compile with specific instances/classes ---
Conv2d with name=conv2d_layer_1564793765312 is setting weights from keras
Weights:
[Tensor(data=[[[[-0.01405657  0.00188509 -0.00080714]
   [-0.0171395   0.00591671 -0.00216555]
   [-0.01256019  0.00688362 -0.00389423]]]


 [[[-0.00193865  0.00779068  0.01467344]
   [-0.01566163  0.01295586 -0.00830487]
   [-0.01288961  0.00373112 -0.01311023]]]], grad=[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], op='None', type=conv2d_layer_1564793765312_kernel), Tensor(data=[0.01127896 0.01435396], grad=[0. 0.], op='None', type=conv2d_layer_1564793765312_bias)]
Tensor(data=[0.01127896 0.01435396], grad=[0. 0.], op='None', type=conv2d_layer_1564793765312_bias)
Model compiled with optimizer Adam and loss MeanSquaredError.
Compile with Adam instance and MSE class: PASSED

--- Integration Test: Deeper CNN End-to-End Gradient Check ---
Calculating numerical gradients for Conv1 Kernel...
Con

.


--- TestSequentialModel: Full Fit, Predict, Evaluate Cycle ---
Conv2d with name=conv2d_layer_1564793749568 is setting weights from keras
Weights:
[Tensor(data=[[[[ 0.00519476  0.0024042   0.00017344]
   [ 0.01279132 -0.00455381 -0.00418927]
   [ 0.00216783  0.00973072 -0.00788295]]]


 [[[-0.0126875  -0.00803957  0.00394394]
   [ 0.00659736  0.01465789  0.01466306]
   [ 0.00507742 -0.00356642 -0.00232578]]]], grad=[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], op='None', type=conv2d_layer_1564793749568_kernel), Tensor(data=[-0.00847501 -0.01748253], grad=[0. 0.], op='None', type=conv2d_layer_1564793749568_bias)]
Tensor(data=[-0.00847501 -0.01748253], grad=[0. 0.], op='None', type=conv2d_layer_1564793749568_bias)
Model compiled with optimizer Adam and loss MeanSquaredError.

Starting model.fit()...
Epoch 1/5
  loss: 0.3377 - val_loss: 0.3532 - 0.01s/epoch                   
Epoch 2/5
  loss: 0.3375 - val_loss: 0.3532 - 0.01s/epoch           

..


Starting model.evaluate() on validation data...
Evaluating...
  2/2 [██████████████████████████████] 100.0%
Evaluation - loss: 0.3532

Starting model.evaluate() on training data...

--- TestSequentialModel: Full Cycle Test PASSED ---

--- TestSequentialModel: Get Parameters ---
Conv2d with name=conv2d_layer_1564793760992 is setting weights from keras
Weights:
[Tensor(data=[[[[ 0.00883893  0.00357537 -0.01084833]
   [ 0.00939469  0.00503097  0.00323461]
   [-0.00792017 -0.01279503 -0.00044195]]]


 [[[ 0.00195865 -0.02343262  0.00559696]
   [-0.00978481  0.00406414 -0.00493411]
   [-0.00842368  0.00245715  0.01567633]]]], grad=[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], op='None', type=conv2d_layer_1564793760992_kernel), Tensor(data=[0.01051109 0.00406368], grad=[0. 0.], op='None', type=conv2d_layer_1564793760992_bias)]
Tensor(data=[0.01051109 0.00406368], grad=[0. 0.], op='None', type=conv2d_layer_1564793760992_bias)
Get Parameters test 

.
----------------------------------------------------------------------
Ran 5 tests in 0.434s

OK


Conv2d with name=conv2d_layer_1564793760272 is setting weights from keras
Weights:
[Tensor(data=[[[[ 0.04967142  0.06476885 -0.02341534]
   [ 0.15792128 -0.04694744 -0.04634177]
   [ 0.02419623 -0.17249178 -0.10128311]]]


 [[[-0.01382643  0.15230299 -0.0234137 ]
   [ 0.07674347  0.054256   -0.04657298]
   [-0.19132802 -0.05622875  0.03142473]]]], grad=[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], op='None', type=conv2d_layer_1564793760272_kernel), Tensor(data=[-0.09080241 -0.14123037], grad=[0. 0.], op='None', type=conv2d_layer_1564793760272_bias)]
Tensor(data=[-0.09080241 -0.14123037], grad=[0. 0.], op='None', type=conv2d_layer_1564793760272_bias)
Conv2d with name=conv2d_layer_1564793760272 is setting weights from keras
Weights:
[Tensor(data=[[[[ 0.04967142  0.06476885 -0.02341534]
   [ 0.15792128 -0.04694744 -0.04634177]
   [ 0.02419623 -0.17239178 -0.10128311]]]


 [[[-0.01382643  0.15230299 -0.0234137 ]
   [ 0.07674347  0.054256   -0.0

<unittest.runner.TextTestResult run=5 errors=0 failures=0>

In [10]:
unittest.TextTestRunner().run(unittest.defaultTestLoader.loadTestsFromTestCase(TestLoadKerasWeights))


--- TestKerasLoad: Multi Conv/Pool ---




F

Loading weights from Keras H5 file: C:\Users\IRFANS~1\AppData\Local\Temp\tmpaus7wzhe\test_keras_weights.h5
  Keras model instance not provided. Relying on custom_layer_name_map or direct name matching.
Successfully loaded weights into 0 custom layers that have parameters.

--- TestKerasLoad: Simple CNN ---


FF
FAIL: test_load_weights_multiple_conv_and_pooling (src.tests.test_load_keras_weights.TestLoadKerasWeights)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Users\Irfan Sidiq\Documents\uni\smt 6\ml\tugas\IF3270_CNN_RNN_LSTM\src\tests\test_load_keras_weights.py", line 129, in test_load_weights_multiple_conv_and_pooling
    self.assert_weights_loaded_correctly(c_conv1, keras_c1_w)
  File "c:\Users\Irfan Sidiq\Documents\uni\smt 6\ml\tugas\IF3270_CNN_RNN_LSTM\src\tests\test_load_keras_weights.py", line 45, in assert_weights_loaded_correctly
    self.assertEqual(len(custom_params), len(keras_layer_weights_list),
AssertionError: 0 != 2 : Param count mismatch for Conv2D layer c_conv1

FAIL: test_load_weights_simple_cnn (src.tests.test_load_keras_weights.TestLoadKerasWeights)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Users\Irfan Sidiq\Documents\uni\sm

Loading weights from Keras H5 file: C:\Users\IRFANS~1\AppData\Local\Temp\tmp6dt5wevf\test_keras_weights.h5
  Keras model instance not provided. Relying on custom_layer_name_map or direct name matching.
Successfully loaded weights into 0 custom layers that have parameters.

--- TestKerasLoad: Skip Missing / Mismatched Names ---
Loading weights from Keras H5 file: C:\Users\IRFANS~1\AppData\Local\Temp\tmpj9effgsr\test_keras_weights.h5
  Keras model instance not provided. Relying on custom_layer_name_map or direct name matching.
Successfully loaded weights into 0 custom layers that have parameters.


<unittest.runner.TextTestResult run=3 errors=0 failures=3>

### **CIFAR-10 Test Case**

In [11]:
(x_train, y_train_raw), (x_test, y_test_raw) = cifar10.load_data()

print(f"Original x_train shape: {x_train.shape}, y_train_raw shape: {y_train_raw.shape}")

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

x_train = np.transpose(x_train, (0, 3, 1, 2))
x_test = np.transpose(x_test, (0, 3, 1, 2))
print(f"Transposed x_train shape: {x_train.shape}")

num_classes = 10
y_train = keras_to_categorical(y_train_raw, num_classes)
y_test = keras_to_categorical(y_test_raw, num_classes)
print(f"One-hot y_train shape: {y_train.shape}")

Original x_train shape: (50000, 32, 32, 3), y_train_raw shape: (50000, 1)
Transposed x_train shape: (50000, 3, 32, 32)
One-hot y_train shape: (50000, 10)


In [12]:
input_channels = x_train.shape[1]

model = Sequential([
    Conv2D(num_kernels=32, kernel_size=3, input_channels=input_channels, padding='same', activation=ReLU, name="conv1"),
    Conv2D(num_kernels=32, kernel_size=3, padding='same', activation=ReLU, name="conv2"),
    MaxPooling2D(pool_size=2, strides=2, name="pool1"),

    Conv2D(num_kernels=64, kernel_size=3, padding='same', activation=ReLU, name="conv3"),
    Conv2D(num_kernels=64, kernel_size=3, padding='same', activation=ReLU, name="conv4"),
    MaxPooling2D(pool_size=2, strides=2, name="pool2"),

    Flatten(name="flatten"),
    Dense(units=512, activation_class=ReLU, name="dense1"),
    Dense(units=num_classes, activation_class=Softmax, name="output_dense")
])

In [13]:
optimizer = Adam(learning_rate=0.001)
loss_function_class = CategoricalCrossEntropy 
model.compile(optimizer=optimizer, loss=loss_function_class)


EPOCHS = 10
BATCH_SIZE = 64

print(f"\nStarting training for {EPOCHS} epochs with batch size {BATCH_SIZE}...")

num_train_samples = x_train.shape[0]
if num_train_samples > 1000:
    val_split_idx = int(num_train_samples * 0.9)
    x_train_fit, y_train_fit = x_train[:val_split_idx], y_train[:val_split_idx]
    x_val_fit, y_val_fit = x_train[val_split_idx:], y_train[val_split_idx:]
    validation_for_fit = (x_val_fit, y_val_fit)
    print(f"Using {x_train_fit.shape[0]} samples for training, {x_val_fit.shape[0]} for validation during fit.")
else:
    print("Using test set for validation during fit (not best practice for final eval).")
    x_train_fit, y_train_fit = x_train, y_train
    validation_for_fit = (x_test, y_test) if x_test is not None else None

history = model.fit(x_train_fit, y_train_fit, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE,
    validation_data=validation_for_fit,
    shuffle=True, 
    verbose=1)

Model compiled with optimizer Adam and loss CategoricalCrossEntropy.

Starting training for 10 epochs with batch size 64...
Using 45000 samples for training, 5000 for validation during fit.
Epoch 1/10
  9/704 [------------------------------] 1.3%

MemoryError: Unable to allocate 16.0 MiB for an array with shape (64, 32, 32, 32) and data type float64

In [ ]:
if history and 'loss' in history:
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history['loss'], label='Training Loss')
    if 'val_loss' in history and not all(np.isnan(history['val_loss'])):
        plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid(True)
    plt.show()
else:
    print("No training history to plot.")

if x_test is not None and y_test is not None:
    print("\nEvaluating on the test set:")
    test_loss = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)
    
    predictions_test_np = model.predict(x_test, batch_size=BATCH_SIZE)
    predicted_classes_test = np.argmax(predictions_test_np, axis=1)
    true_classes_test = np.argmax(y_test, axis=1)
    
    accuracy_test = np.mean(predicted_classes_test == true_classes_test)
    print(f"Test Accuracy: {accuracy_test * 100:.2f}%")
else:
    print("No test data available for final evaluation.")